## **Riot API Tutorial (2025-02-25)**

*This sandbox follows the youtube tutorial series by Beora ([Youtube Link](https://www.youtube.com/watch?v=A5c0yfEiYjQ&list=PLnnM9dgSdUvIe0MRJcjbzBaJWFHNJaOh1)). It is meant to give a basic Introduction to the Tools Necessary to Get Started working on Projects utilising Riot's API.*

### **1. Establishing Credentials**

In [9]:
from dotenv import load_dotenv
import os

load_dotenv()

print(os.environ.get('riot_api_key'))

RGAPI-d0d71486-25a2-4155-92a3-38402a19a68d


In [10]:
import pygsheets

service_account = pygsheets.authorize(service_account_file='JSONs\\spreadsheet-automator-452009-1b2fd6310821.json')

sheet = service_account.open_by_url('https://docs.google.com/spreadsheets/d/1F38cGIAQtiPwX0CjVb742iqwnnMW8Lh2Exy-SRP_BrM/edit?usp=sharing')

In [11]:
test = sheet.worksheet_by_title('Test')

In [12]:
test_df = test.get_as_df()

In [13]:
test_df['random_numbers'] = 1

In [14]:
test.set_dataframe(test_df, 'A1', copy_head=True)

### **2. Interacting with the API**

In [15]:
from dotenv import load_dotenv
load_dotenv()

import os
import requests

gameName = 'LordVoItage'
tagLine = 'EUW'
api_key = os.environ.get('riot_api_key')

def get_puuid(gameName=None, tagLine=None, api_key=None):
    link = f'https://europe.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}%20/{tagLine}?api_key={api_key}'

    response = requests.get(link)
    response.json()

    return response.json()['puuid']

def get_name_and_tag(puuid=None, api_key=None):
    link = f'https://europe.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}'

    response = requests.get(link)
    response.json()

    gameName = response.json()['gameName']
    tagLine = response.json()['tagLine']

    return f'{gameName}#{tagLine}'

In [16]:
get_puuid(gameName, tagLine, api_key)

'sYgaAEq5FqgCqn7iIse8zzNHhmQ8CbQP4YVR-Lyg5ZQT_UvjRriqWzHTzHbG2pO_k74mXuHuQw5rJw'

In [17]:
get_name_and_tag(puuid=get_puuid(gameName, tagLine, api_key), api_key=api_key)

'LordVoItage#EUW'

### **3. Retrieving the SoloQ Ladder**

In [18]:
response = requests.get('https://euw1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=RGAPI-d0d71486-25a2-4155-92a3-38402a19a68d')

puuid = response.json()['entries'][0]['puuid']

summoner_name = get_name_and_tag(puuid, api_key)

summoner_name

'G2 SkewMond#3327'

In [19]:
import pandas as pd

def get_ladder(top=None, api_key=None):
    root = 'https://euw1.api.riotgames.com/'
    chall = 'lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    gmaster = 'lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'
    master = 'lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'

    chall_response = requests.get(root + chall + '?api_key=' + api_key)
    gmaster_response = requests.get(root + gmaster + '?api_key=' + api_key)
    master_response = requests.get(root + master + '?api_key=' + api_key)

    chall_df = pd.DataFrame(chall_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    gmaster_df = pd.DataFrame(gmaster_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    master_df = pd.DataFrame(master_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    ladder = pd.concat([chall_df, gmaster_df, master_df], keys=['Challenger', 'Grandmaster', 'Master'])[:top].reset_index(drop=True)

    ladder = ladder.drop(columns='rank').reset_index(drop=False).rename(columns={'index': 'rank'})
    ladder['rank'] += 1

    return ladder

In [20]:
ladder = get_ladder(top=10, api_key=api_key)

In [21]:
ladder['summoner_name'] = ladder['puuid'].apply(lambda x: get_name_and_tag(x, api_key))

In [22]:
ladder

,rank,summonerId,puuid,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak,summoner_name
0,1,aWgR5OW8ruiR-BZy2CmyE01u6iWSsiZrTO_JJ5tVK01_oriX,vGfPE8LWdIKRLLC0huTNYNKwSJ4dyesDd4J5tVy8LSUNUl...,1969,175,122,True,False,False,False,G2 SkewMond#3327
1,2,fpCdtOdci7tnvJDPdNC1U6KDRs_lq64Op6mqeUyKVLlzrd...,ZhxdGQCRKO7k1FUjbQfGO2Ln-Dxl4n9oA9txYN2rqhoyis...,1840,226,175,True,False,False,False,J1HUIV#000
2,3,iTeb8_FNg2J4aDakvghhJUbm-002iasywLms0iUF_4PM7n...,HAJ71n_egQn7Dkfw9y6CXPQ9N4Cnrdxm1j5RIpbIjaieL2...,1737,226,146,True,False,False,False,Ivory#EfEf
3,4,8G_VDEDjVDXP9mXgESWI_9bILOaUnPHsrhB6PJnwbqse97o,Az2f_S0MvTr-7m97Wsiyj_i0ZGKSPrG9hIXJv5tSYYG4sg...,1723,121,72,True,False,False,False,Odysseus#131
4,5,29HbMrSHNq2orYPCDPuaWiB5YXyajxlVCZCg9XNjNQcAKOw,BA8fVwA36g9KvqQLc5DRTE1mpebZsUdPHKxwzlfjII7-zc...,1698,168,126,True,False,False,False,G2 Caps#1323
5,6,wTb2njNhOR_7bDDj_RMlK9jBndAScbeZZLpL472wQN0caIto,QgQADXhXsMxHlpciCId-0PYgPwGFtdCQ6malsPbwjoGCUp...,1697,146,101,True,False,False,False,lpl dna#12103
6,7,0fPtqO_Ptj8EiqYaGlbv5Z6wZ6D_7oYqnMJmeTlNF7FIeQw,QtOWvEaM-3H3cFdQLbH3qILSwZDBbcQOaT4-aTSNEZWjVV...,1673,147,106,True,False,False,False,G2 Labrov#8085
7,8,KHBIZVoYC50_YO0oSJ5rXtBErZL8kIozDAXJIiYlWcJkdCGt,7PHuKgmARX_YeXzyRninl-SriUyX9E4PKa44ckoLdv630B...,1627,111,74,True,False,False,False,KC NEXT ADKING#EUW
8,9,ujQ7AWE6GNKTVHk1wkjX4agW3qBdd3t82CXBWBruAB5jcS9k,wQ3sHQG8e56XQa_zEUsnMOVihPr4v2pnsb2HkZjuFfWleY...,1601,129,85,True,False,False,False,TwTv Phantasm#103
9,10,9BSiPqVSq27Var4YFNZaqabQsXODdH2XSi02ZzNCBMhGo5Y,Fn2pIdf1zFAtFPBUicQdoztmbEDoUxV1W8pnc6Sc_LbeCl...,1587,146,107,True,False,False,False,G2 Hans Sama#12838


### **4. Obtaining Match Histories**

In [38]:
def get_match_history(puuid=None, api_key=None, start=0, count=20):

    root = 'https://europe.api.riotgames.com/'
    endpoint = f'lol/match/v5/matches/by-puuid/{puuid}/ids'
    query_params = f'?type=ranked&start={start}&count={count}'

    response = requests.get(root + endpoint + query_params + '&api_key=' + api_key)

    return response.json()

def get_match_data_from_id(matchId=None, api_key=None):

    root = 'https://europe.api.riotgames.com/'
    endpoint = f'lol/match/v5/matches/{matchId}'

    response = requests.get(root + endpoint + '?api_key=' + api_key)

    return response.json()


In [ ]:
puuid = get_puuid(gameName, tagLine, api_key)
matchId = get_match_history(puuid, api_key, start=0, count=100)[0]

get_match_data_from_id(matchId, api_key)

{'metadata': {'dataVersion': '2',
  'matchId': 'EUW1_7317675815',
  'participants': ['9yECzY_qZRykY05W1qz37r4loWBQjA0gF3rNuEuqqHmtloyDeg2I4LpTm5sT9L2uMp-oIjB8_BfNdg',
   'U7aHYNfy8KKaGl46wvJA24ag-TYUOcGin_ARtlniAnxpjdpK1NZvdKUcdsEb1CYWrKBHXFT4zNQXPQ',
   '6hBf0MBsJkWo4vTDXJ6WM2u024Ww04JSMDsD06xf9zR4xiT7O7Pg59ZT-3O644mM4-Fl_a2NaCHYhQ',
   'NEzHGIamYgeOxvlJSxCjhYjK89xHCvXlueQyJVVQQwFL9EA5XN0Tjqb6CSNZxiwezhrGW4KBUAGSOw',
   '7rnquCJUtUU17bYV9tweVpi1HzfvdICy0csPEMcTqgwjwPB2ra8MaTBU5Ci4Goch51zHROkLTCc_uQ',
   'H_JK6bXTiLE3NiMXecRMaOIE5raLhzWkJ_manTPFxHhoMECtc68UQKDURkNLNvDo68UU9pEUkMHEog',
   'sYgaAEq5FqgCqn7iIse8zzNHhmQ8CbQP4YVR-Lyg5ZQT_UvjRriqWzHTzHbG2pO_k74mXuHuQw5rJw',
   '7J7ikEoyDFQMEQeNojtzE_LIjeW2qJZXirGZuZn_olvip6yaRxyeHUcuG4YJxt22UZMqTufcbf0hMg',
   'On8aKn4RxhweTnG6bsL_WiJuXoJRpLdEToYTT0L7_3xqz-uOPKgt-f7SaBKhLvm12p_y4YyOgU9inw',
   'IvzgjokXfwX4uQv52XMWmIAKYdkYF38QkoNi7bdkSbfNqxFyAZVw4lQIz6yk6Hj2ZxcCkW_lYXMnzw']},
 'info': {'endOfGameResult': 'GameComplete',
  'gameCreation': 174

### **5. Processing SoloQ Matches**